## Bài 4: ES

### ES cho bài toán Sphere

In [1]:
import numpy as np
import random
import math

In [2]:
def sphere(x):
    return np.sum(x * x)

def initialize_population(mu, n_dim, bounds, init_sigma, rng):
    low, high = bounds
    X = rng.uniform(low, high, size=(mu, n_dim))
    Sig = np.full((mu, n_dim), init_sigma, dtype=float)
    return X, Sig

def recombine(X, Sig):
    x_mean = np.mean(X, axis=0)
    sigma_mean = np.exp(np.mean(np.log(Sig), axis=0))
    return x_mean, sigma_mean


def mutate(x_mean, sigma_mean, lam, bounds, tau, tau_prime, rng, sigma_min=1e-12):
    low, high = bounds
    n = len(x_mean)

    children_X = np.empty((lam, n))
    children_S = np.empty((lam, n))

    for k in range(lam):
        N_global = rng.normal()
        N_local = rng.normal(size=n)
        sigma_child = sigma_mean * np.exp(tau_prime * N_global + tau * N_local)
        sigma_child = np.clip(sigma_child, sigma_min, None)

        x_child = x_mean + sigma_child * rng.normal(size=n)
        x_child = np.clip(x_child, low, high)

        children_X[k] = x_child
        children_S[k] = sigma_child

    return children_X, children_S

def select(X, Sig, fitness, children_X, children_S, children_fit, mu, variant):
    if variant == "comma":
        sel = np.argsort(children_fit)[:mu]
        return children_X[sel], children_S[sel], children_fit[sel]
    elif variant == "plus":
        X_stack = np.vstack([X, children_X])
        S_stack = np.vstack([Sig, children_S])
        f_stack = np.concatenate([fitness, children_fit])
        sel = np.argsort(f_stack)[:mu]
        return X_stack[sel], S_stack[sel], f_stack[sel]
    else:
        raise ValueError("variant phải là 'comma' hoặc 'plus'.")

def es_optimize(
    f,
    n_dim=20,
    mu=10,
    lam=70,
    variant="comma",
    bounds=(-5.0, 5.0),
    max_evals=1000,
    target_f=1e-10,
    seed=42,
    init_sigma=0.3,
    verbose_every=0,
):
    rng = np.random.default_rng(seed)
    tau_prime = 1.0 / np.sqrt(2.0 * n_dim)
    tau = 1.0 / np.sqrt(2.0 * np.sqrt(n_dim))
    X, Sig = initialize_population(mu, n_dim, bounds, init_sigma, rng)
    fitness = np.array([f(x) for x in X])
    evals = mu
    best_idx = int(np.argmin(fitness))
    best_x = X[best_idx].copy()
    best_f = float(fitness[best_idx])
    history = [best_f]
    while evals < max_evals and best_f > target_f:
        order = np.argsort(fitness)
        X, Sig, fitness = X[order], Sig[order], fitness[order]
        x_mean, sigma_mean = recombine(X, Sig)
        children_X, children_S = mutate(
            x_mean, sigma_mean, lam, bounds, tau, tau_prime, rng)
        children_fit = np.array([f(xc) for xc in children_X])
        evals += lam
        X, Sig, fitness = select(
            X, Sig, fitness, children_X, children_S, children_fit, mu, variant)
        idx = int(np.argmin(fitness))
        if fitness[idx] < best_f:
            best_f = float(fitness[idx])
            best_x = X[idx].copy()

        history.append(best_f)

        if verbose_every and evals % verbose_every == 0:
            print(f"evals={evals:7d}  best_f={best_f:.3e}")
    return best_x, best_f, history

In [3]:
if __name__ == "__main__":
    mu, lam, n_dim = 10, 70, 30

    print("== (μ, λ)-ES trên Sphere ==")
    bx, bf, hist = es_optimize(
        sphere, n_dim=n_dim, mu=mu, lam=lam, variant="comma")
    print(f"(μ,λ)-ES: best_f = {bf:.3e}")

    print("\n== (μ + λ)-ES trên Sphere ==")
    bx2, bf2, hist2 = es_optimize(
        sphere, n_dim=n_dim, mu=mu, lam=lam, variant="plus")
    print(f"(μ+λ)-ES: best_f = {bf2:.3e}")

== (μ, λ)-ES trên Sphere ==
(μ,λ)-ES: best_f = 1.180e+00

== (μ + λ)-ES trên Sphere ==
(μ+λ)-ES: best_f = 9.706e-01
